In [ ]:
!pip install -U bitsandbytes

In [73]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
)
import torch
import json
import re

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [33]:
# del model, tokenizer
# torch.cuda.empty_cache()
# import gc; gc.collect()

0

In [34]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct", device_map="auto", torch_dtype=torch.bfloat16,
    quantization_config= BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [35]:
model.get_memory_footprint()

1122135296

In [55]:
messages = {
    "conversation": [
        {
            "content": "You are helpful assistant, you provide helping tools. You have all right to use them if required.",
            "role": "system"
        },
        {
            "content": "Use tools if required, don't ask questions to the user.",
            "role": "system"
        },
        {
            "role": "user",
            "content": "What's the weather today in Casablanca?"
        }
    ],
    "model": "qwen2.5:1.5b",
    "response_format": {
        "type": "json_schema",
        "json_schema": {
            "name": "final_output",
            "strict": True,
            "schema": {
                "properties": {
                    "city": {
                        "default": "Rabat",
                        "description": "this field represents the city name.",
                        "title": "City",
                        "type": "string"
                    },
                    "weather": {
                        "default": "Sunny",
                        "description": "this field represents, the weather sunny ...",
                        "title": "Weather",
                        "type": "string"
                    }
                },
                "title": "WeatherModel",
                "type": "object",
                "additionalProperties": False,
                "required": [
                    "city",
                    "weather"
                ]
            }
        }
    },
    "stream": False,
    "tools": [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Gives the weather of given city as arguments.",
                "parameters": {
                    "properties": {
                        "city": {
                            "title": "City",
                            "type": "string"
                        }
                    },
                    "required": [
                        "city"
                    ],
                    "title": "get_weather_args",
                    "type": "object",
                    "additionalProperties": False
                }
            }
        }
    ]
}

In [56]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}
    {%- endif %}
{%- endif %}

In [57]:
chat_template = tokenizer.apply_chat_template(**messages, tokenize=False, add_generation_prompt=True)
print(chat_template)

<|im_start|>system
You are helpful assistant, you provide helping tools. You have all right to use them if required.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_weather", "description": "Gives the weather of given city as arguments.", "parameters": {"properties": {"city": {"title": "City", "type": "string"}}, "required": ["city"], "title": "get_weather_args", "type": "object", "additionalProperties": false}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>system
Use tools if required, don't ask questions to the user.<|im_end|>
<|im_start|>user
What's the weather today in Casablanca?<|im_end|>
<|im_start|>assistant



In [58]:
inputs_template = tokenizer(chat_template, return_tensors="pt").to(model.device)
print(inputs_template)

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,  10950,  17847,     11,    498,
           3410,  10476,   7375,     13,   1446,    614,    678,   1290,    311,
            990,   1105,    421,   2567,    382,      2,  13852,    271,   2610,
           1231,   1618,    825,    476,    803,   5746,    311,   7789,    448,
            279,   1196,   3239,    382,   2610,    525,   3897,    448,    729,
          32628,   2878,    366,  15918,   1472,  15918,     29,  11874,   9492,
            510,     27,  15918,    397,   4913,   1313,    788,    330,   1688,
            497,    330,   1688,    788,   5212,    606,    788,    330,    455,
          69364,    497,    330,   4684,    788,    330,     38,   1886,    279,
           9104,    315,   2661,   3283,    438,   5977,  10465,    330,  13786,
            788,   5212,  13193,    788,   5212,   8926,    788,   5212,   2102,
            788,    330,  12730,    497,    330,   1313,    788,    330,    917,
           920

In [61]:
outputs = model.generate(**inputs_template, max_new_tokens=512)

In [153]:
response = tokenizer.decode(outputs[0][inputs_template["input_ids"].shape[1]:])
print(response)

<tool_call>
{"name": "get_weather", "arguments": {"city": "Casablanca"}}
</tool_call><|im_end|>


In [164]:
func_args = json.loads(response.split("</tool_call>")[0].split("<tool_call>")[1])
func_args

{'name': 'get_weather', 'arguments': {'city': 'Casablanca'}}

In [168]:
def get_weather(city: str, **kwargs) -> str:
    """Gives the weather of given city as arguments."""
    return f"The weather in {city} is sunny"

In [170]:
get_weather(**func_args["arguments"])

'The weather in Casablanca is sunny'